In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning')
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning/src')
sys.path.insert(0, '/home/er647/projects/feature-wise-active-learning/notebooks')

In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb
import datetime
from utils import *
# from _utils import filter_df_on_conditions, make_subplots
import seaborn as sns
from functools import partial

In [4]:
datetime.datetime.now(datetime.timezone.utc).isoformat()

'2024-02-20T20:25:10.526484+00:00'

In [5]:
# wandb.init(project="fwal",entity="evangeorgerex")
api = wandb.Api(timeout=60) #, entity='evangeorgerex')

In [133]:
entity = "evangeorgerex"
project_name = "fwal"
sweep_id = "bshht322"

# Fetch the sweep
api = wandb.Api()
sweep = api.sweep(f"{entity}/{project_name}/{sweep_id}")


In [134]:
run = sweep.runs[0]

In [136]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: evangeorgerex (l65). Use `wandb login --relogin` to force relogin


True

In [142]:
history_data = []

# Iterate over the scan_history generator
for metrics in run.scan_history():
    history_data.append(metrics)

# Convert the list of dictionaries into a DataFrame
df1 = pd.DataFrame(history_data)
    
history_data = []
# Iterate over the scan_history generator
for metrics in run.scan_history():
    history_data.append(metrics)

# Convert the list of dictionaries into a DataFrame
df2 = pd.DataFrame(history_data)

df1.equals(df2)

True

In [143]:

# Data structure to hold all sweep data
sweep_data = []

# Iterate over each run in the sweep
for run in sweep.runs:
    # Fetching run summary and config data
    run_summary = run.summary._json_dict
    run_config = {k: v for k, v in run.config.items() if not k.startswith('_')}
    
    history_data = []
    
    # Iterate over the scan_history generator
    for metrics in run.scan_history():
        history_data.append(metrics)

    # Convert the list of dictionaries into a DataFrame
    history_df = pd.DataFrame(history_data)

   
    # Store run data in a dictionary and append to the list
    run_data = {
        "id": run.id,
        "summary": run_summary,
        "config": run_config,
        "history": history_df.to_dict() if 'history_df' in locals() else None  # Uncomment if history data is added
    }
    sweep_data.append(run_data)



In [156]:
data = []

# Assuming sweep.runs is a list of runs you want to process

for run in sweep_data:
    run_id = run['id']
    run_summary = run['summary']
    run_config = run['config']
    history_df = pd.DataFrame.from_dict(run['history'])
    
    
    history_df['dataset'] = run_config['dataset']
    history_df['id'] = run_id
    
    history_df['bestmodel_valid/balanced_accuracy'] = run_summary['bestmodel_valid/balanced_accuracy']
    history_df['bestmodel_valid/mask'] = run_summary['best_mask'] 
    
    rounded_str_list = [f"{round(num, 2):.2f}" for num in run_summary['best_mask_parameters']]

    # Convert the list of strings to a single string representation of the list
    final_str = "[" + ", ".join(rounded_str_list) + "]"

    history_df['bestmodel_valid/mask_parameters'] =  final_str
    
    # Append the augmented DataFrame to our list
    data.append(history_df)

# If you have multiple runs and want to combine them, you can do so. Here's how to concatenate all the DataFrames:
combined_df = pd.concat(data, ignore_index=True)


In [158]:
combined_df['max_balanced_accuracy'] = combined_df.groupby('id')['masks/valid_metrics.balanced_accuracy'].transform(max)

# Step 2: Filter to keep only rows where 'masks/valid_metrics.balanced_accuracy' matches the max for that 'id'
max_accuracy_rows = combined_df[combined_df['masks/valid_metrics.balanced_accuracy'] == combined_df['max_balanced_accuracy']]

# Drop the auxiliary column 'max_balanced_accuracy' if it's no longer needed
max_accuracy_rows = max_accuracy_rows.drop(columns=['max_balanced_accuracy'])

max_accuracy_rows.groupby(['dataset','id','masks/mask']).agg({
    # 'id': 'first',
    'masks/valid_metrics.balanced_accuracy': 'mean',
    'bestmodel_valid/balanced_accuracy': 'mean',
    'bestmodel_valid/mask': 'first',
    'bestmodel_valid/mask_parameters':'first'
    }).dropna()

masks/valid_metrics.balanced_accuracy  \
dataset                          id       masks/mask                                           
simple_trig_synth                amkig6nm 00001                                     0.910500   
                                 bufecrzp 11110                                     0.952000   
                                 qem7n8jo 10010                                     0.944000   
summed_squares_exponential_synth mziy0co9 10000001000                               0.661093   
                                 t5lbbiud 00001010001                               0.685747   
                                 zxgd857c 10010001000                               0.659019   
trigonometric_polynomial_synth   b229fw35 00000000001                               0.991335   
                                 l7wzqxx8 10010001000                               0.834366   
                                 xqwnjkjm 10000001000                               0.975884   

                                                       bestmodel_valid/balanced_accuracy  \
dataset                          id       masks/mask                                       
simple_trig_synth                amkig6nm 00001                                 0.906805   
                                 bufecrzp 11110                                 0.903861   
                                 qem7n8jo 10010                                 0.955232   
summed_squares_exponential_synth mziy0co9 10000001000                           0.500000   
                                 t5lbbiud 00001010001                           0.500000   
                                 zxgd857c 10010001000                           0.500000   
trigonometric_polynomial_synth   b229fw35 00000000001                           0.935392   
                                 l7wzqxx8 10010001000                           0.500000   
                                 xqwnjkjm 10000001000                           0.958532   

                                                      bestmodel_valid/mask  \
dataset                          id       masks/mask                         
simple_trig_synth                amkig6nm 00001                      00001   
                                 bufecrzp 11110                      00010   
                                 qem7n8jo 10010                      10010   
summed_squares_exponential_synth mziy0co9 10000001000          00000000000   
                                 t5lbbiud 00001010001          00000000000   
                                 zxgd857c 10010001000          00000000000   
trigonometric_polynomial_synth   b229fw35 00000000001          00000000001   
                                 l7wzqxx8 10010001000          00000000000   
                                 xqwnjkjm 10000001000          10000001000   

                                                                         bestmodel_valid/mask_parameters  
dataset                          id       masks/mask                                                      
simple_trig_synth                amkig6nm 00001                       [-1.40, -1.58, -1.46, -2.19, 0.02]  
                                 bufecrzp 11110                       [-0.83, -0.99, -0.84, 0.01, -1.60]  
                                 qem7n8jo 10010                        [0.04, -0.92, -3.09, 0.16, -2.01]  
summed_squares_exponential_synth mziy0co9 10000001000  [-1.89, -2.28, -3.26, -2.23, -2.73, -2.75, -2....  
                                 t5lbbiud 00001010001  [-2.15, -2.27, -2.39, -2.67, -1.99, -2.47, -2....  
                                 zxgd857c 10010001000  [-2.15, -2.25, -2.26, -2.22, -2.44, -2.09, -2....  
trigonometric_polynomial_synth   b229fw35 00000000001  [-1.07, -1.54, -1.61, -2.26, -0.45, -1.81, -0....  
                                 l7wzqxx8 10010001000  [-2.15, -2.25, -2.25, -2.23, -2.33, -2.01, -2....  
                                 xqwnjkjm 10000001000  [

In [145]:
combined_df['max_balanced_accuracy'] = combined_df.groupby('id')['masks/valid_metrics.balanced_accuracy'].transform(max)

# Step 2: Filter to keep only rows where 'masks/valid_metrics.balanced_accuracy' matches the max for that 'id'
max_accuracy_rows = combined_df[combined_df['masks/valid_metrics.balanced_accuracy'] == combined_df['max_balanced_accuracy']]

# Drop the auxiliary column 'max_balanced_accuracy' if it's no longer needed
max_accuracy_rows = max_accuracy_rows.drop(columns=['max_balanced_accuracy'])

max_accuracy_rows.groupby(['dataset','id','masks/mask']).agg({
    # 'id': 'first',
    'masks/valid_metrics.balanced_accuracy': 'mean',
    'bestmodel_valid/balanced_accuracy': 'mean',
    'bestmodel_valid/mask': 'first',
    'bestmodel_valid/mask_parameters':'first'
    }).dropna()

masks/valid_metrics.balanced_accuracy  \
dataset                          id       masks/mask                                           
simple_trig_synth                amkig6nm 00001                                     0.910500   
                                 bufecrzp 11110                                     0.952000   
                                 qem7n8jo 10010                                     0.944000   
summed_squares_exponential_synth mziy0co9 10000001000                               0.661093   
                                 t5lbbiud 00001010001                               0.685747   
                                 zxgd857c 10010001000                               0.659019   
trigonometric_polynomial_synth   b229fw35 00000000001                               0.991335   
                                 l7wzqxx8 10010001000                               0.834366   
                                 xqwnjkjm 10000001000                               0.975884   

                                                       bestmodel_valid/balanced_accuracy  \
dataset                          id       masks/mask                                       
simple_trig_synth                amkig6nm 00001                                 0.906805   
                                 bufecrzp 11110                                 0.903861   
                                 qem7n8jo 10010                                 0.955232   
summed_squares_exponential_synth mziy0co9 10000001000                           0.500000   
                                 t5lbbiud 00001010001                           0.500000   
                                 zxgd857c 10010001000                           0.500000   
trigonometric_polynomial_synth   b229fw35 00000000001                           0.935392   
                                 l7wzqxx8 10010001000                           0.500000   
                                 xqwnjkjm 10000001000                           0.958532   

                                                      bestmodel_valid/mask  \
dataset                          id       masks/mask                         
simple_trig_synth                amkig6nm 00001                      00001   
                                 bufecrzp 11110                      00010   
                                 qem7n8jo 10010                      10010   
summed_squares_exponential_synth mziy0co9 10000001000          00000000000   
                                 t5lbbiud 00001010001          00000000000   
                                 zxgd857c 10010001000          00000000000   
trigonometric_polynomial_synth   b229fw35 00000000001          00000000001   
                                 l7wzqxx8 10010001000          00000000000   
                                 xqwnjkjm 10000001000          10000001000   

                                                                         bestmodel_valid/mask_parameters  
dataset                          id       masks/mask                                                      
simple_trig_synth                amkig6nm 00001                       [-1.40, -1.58, -1.46, -2.19, 0.02]  
                                 bufecrzp 11110                       [-0.83, -0.99, -0.84, 0.01, -1.60]  
                                 qem7n8jo 10010                        [0.04, -0.92, -3.09, 0.16, -2.01]  
summed_squares_exponential_synth mziy0co9 10000001000  [-1.89, -2.28, -3.26, -2.23, -2.73, -2.75, -2....  
                                 t5lbbiud 00001010001  [-2.15, -2.27, -2.39, -2.67, -1.99, -2.47, -2....  
                                 zxgd857c 10010001000  [-2.15, -2.25, -2.26, -2.22, -2.44, -2.09, -2....  
trigonometric_polynomial_synth   b229fw35 00000000001  [-1.07, -1.54, -1.61, -2.26, -0.45, -1.81, -0....  
                                 l7wzqxx8 10010001000  [-2.15, -2.25, -2.25, -2.23, -2.33, -2.01, -2....  
                                 xqwnjkjm 10000001000  [

masks/valid_metrics.balanced_accuracy  \
dataset                          id       masks/mask                                           
simple_trig_synth                amkig6nm 00001                                     0.910500   
                                 bufecrzp 10010                                     0.932500   
                                 qem7n8jo 10010                                     0.944000   
summed_squares_exponential_synth mziy0co9 00000000000                               0.500000   
                                 t5lbbiud 00001000001                               0.685363   
                                 zxgd857c 00000000000                               0.500000   
trigonometric_polynomial_synth   b229fw35 00000000001                               0.991335   
                                 l7wzqxx8 11010001000                               0.684023   
                                 xqwnjkjm 10010011000                               0.844466   

                                                       bestmodel_valid/balanced_accuracy  \
dataset                          id       masks/mask                                       
simple_trig_synth                amkig6nm 00001                                 0.906805   
                                 bufecrzp 10010                                 0.903861   
                                 qem7n8jo 10010                                 0.955232   
summed_squares_exponential_synth mziy0co9 00000000000                           0.500000   
                                 t5lbbiud 00001000001                           0.500000   
                                 zxgd857c 00000000000                           0.500000   
trigonometric_polynomial_synth   b229fw35 00000000001                           0.935392   
                                 l7wzqxx8 11010001000                           0.500000   
                                 xqwnjkjm 10010011000                           0.958532   

                                                      bestmodel_valid/mask  \
dataset                          id       masks/mask                         
simple_trig_synth                amkig6nm 00001                      00001   
                                 bufecrzp 10010                      00010   
                                 qem7n8jo 10010                      10010   
summed_squares_exponential_synth mziy0co9 00000000000          00000000000   
                                 t5lbbiud 00001000001          00000000000   
                                 zxgd857c 00000000000          00000000000   
trigonometric_polynomial_synth   b229fw35 00000000001          00000000001   
                                 l7wzqxx8 11010001000          00000000000   
                                 xqwnjkjm 10010011000          10000001000   

                                                                         bestmodel_valid/mask_parameters  
dataset                          id       masks/mask                                                      
simple_trig_synth                amkig6nm 00001                       [-1.40, -1.58, -1.46, -2.19, 0.02]  
                                 bufecrzp 10010                       [-0.83, -0.99, -0.84, 0.01, -1.60]  
                                 qem7n8jo 10010                        [0.04, -0.92, -3.09, 0.16, -2.01]  
summed_squares_exponential_synth mziy0co9 00000000000  [-1.89, -2.28, -3.26, -2.23, -2.73, -2.75, -2....  
                                 t5lbbiud 00001000001  [-2.15, -2.27, -2.39, -2.67, -1.99, -2.47, -2....  
                                 zxgd857c 00000000000  [-2.15, -2.25, -2.26, -2.22, -2.44, -2.09, -2....  
trigonometric_polynomial_synth   b229fw35 00000000001  [-1.07, -1.54, -1.61, -2.26, -0.45, -1.81, -0....  
                                 l7wzqxx8 11010001000  [-2.15, -2.25, -2.25, -2.23, -2.33, -2.01, -2....  
                                 xqwnjkjm 10010011000  [